In [1]:
import pandas as pd
import traceback
import os,sys

sys.path.append(os.path.abspath(os.path.join('../scripts')))
from MetaCreate import MetaCreate
from AudioManipulator import AudioManipulator
from preprocess_uitls import pad_audio_files, featurize
from utils import char_map,index_map, int_sequence_to_text ,text_to_int_sequence


import librosa
import matplotlib.pyplot as plt
# from mpl_toolkits.axes_grid1 import make_axes_locatable

    
# from keras.utils.vis_utils import plot_model


import _pickle as pickle
from numpy.lib.stride_tricks import as_strided

from Models import cnn_rnn_model,simple_rnn,model_2,bidirectional_rnn_model_gpu,bidirectional_rnn_model
# from keras.callbacks import ModelCheckpoint   

In [2]:
data=pd.read_json('../artifacts/meta.json')
data.drop(data[data["text"].str.contains("""[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~1234567890]""", na=False)].index,inplace=True)
with open('../artifacts/features.pkl','rb') as tel:
    features=pickle.load(tel)
    tel.close()

In [3]:
valid_data=pd.read_json("../artifacts/valid_meta.json")
valid_data.drop(valid_data[valid_data["text"].str.contains("""[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~1234567890]""", na=False)].index,inplace=True)
with open('../artifacts/valid_features.pkl','rb') as val:
    valid_features=pickle.load(val)
    val.close()

# Augmentation


In [4]:
from preprocess_uitls import pad_audio_files,featurize,pitch_audio,shift

In [29]:
import preprocess_uitls
import importlib
importlib.reload(preprocess_uitls)

<module 'preprocess_uitls' from 'd:\\same\\Documents\\10Acadamy\\Week 4\\System\\Week-4\\scripts\\preprocess_uitls.py'>

In [5]:
data["audio"]=pad_audio_files(data["path"],98400)

In [6]:
valid_data["audio"]=pad_audio_files(valid_data["path"],98400)

In [8]:
pitched=pitch_audio(data["audio"],-2,16000)

In [9]:
pitched_valid=pitch_audio(valid_data["audio"],-1,16000)

In [10]:
pitched_features=featurize(pitched,16000)

In [11]:
with open('../artifacts/pitched_features.pickle', 'wb') as p:
        pickle.dump(pitched_features, p)

In [12]:
pitched_valid_features=featurize(pitched_valid,16000)

In [14]:
pitched=""
pitched_valid=""

In [13]:
with open('../artifacts/pitched_valid_features.pickle', 'wb') as v:
        pickle.dump(pitched_valid_features, v)

In [15]:
shifted=shift(data["audio"],"right",1,16000)

In [16]:
shifted_valid=shift(valid_data["audio"],"right",1,16000)

In [17]:
shifted_features=featurize(shifted,16000)

In [18]:
shifted_valid_features=featurize(shifted_valid,16000)

In [19]:
with open('../artifacts/shifted_features.pickle', 'wb') as v:
        pickle.dump(shifted_features, v)

In [20]:
with open('../artifacts/shifted_valid_features.pickle', 'wb') as vs:
        pickle.dump(shifted_valid_features, vs)

In [21]:
shifted=""
shifted_valid=""

In [22]:
total_text=[]
total_features=[]
total_valid_text=[]
total_valid_features=[]

In [23]:
import numpy as np

In [24]:
total_text=data["text"].tolist()+data["text"].tolist()+data["text"].tolist()
total_features=features+pitched_features+shifted_features
total_valid_text=valid_data["text"].tolist()+valid_data["text"].tolist()+valid_data["text"].tolist()
total_valid_features=valid_features+pitched_valid_features+shifted_valid_features

In [62]:
len(total_valid_features)

5970

In [25]:
data=[]
valid_data=[]

In [26]:
shifted_valid_features,shifted_features,pitched_features,pitched_valid_features=[],[],[],[]

In [27]:
import mlflow
import mlflow.tensorflow

In [28]:
from Batch import Batch

In [29]:
cnn_rnn = cnn_rnn_model(input_dim=26, # change to 13 if you would like to use MFCC features
                        filters=250,
                        kernel_size=4, 
                        conv_stride=1,
                        conv_border_mode='same',
                        units=200,output_dim=29)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 26)]        0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 250)         26250     
_________________________________________________________________
bn_conv_1d (BatchNormalizati (None, None, 250)         1000      
_________________________________________________________________
rnn (SimpleRNN)              (None, None, 200)         90200     
_________________________________________________________________
batch_normalization (BatchNo (None, None, 200)         800       
_________________________________________________________________
time_distributed (TimeDistri (None, None, 29)          5829      
_________________________________________________________________
softmax (Activation)         (None, None, 29)          0     

In [30]:

MODEL_NAME = "cnn_rnn_augmented"
EPOCHS=50
MINI_BATCH_SIZE = 200
FEATURES_LIST=total_features

TEXT_LIST=total_text

FEATURES_VALID_LIST=total_valid_features

TEXT_VALID_LIST=total_valid_text

audio_gen = Batch(FEATURES_LIST,FEATURES_VALID_LIST,TEXT_LIST,TEXT_VALID_LIST,MINI_BATCH_SIZE)

In [31]:
from Train import train

In [32]:
import mlflow
import mlflow.tensorflow

In [33]:
mlflow.tensorflow.autolog()
mlflow.set_experiment("CNN RNN Augmented Data")
train(audio_gen, input_to_softmax=cnn_rnn, model_name=MODEL_NAME, epochs=EPOCHS, minibatch_size=MINI_BATCH_SIZE)

mlflow.end_run()

D:\Users\same\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021/08/11 03:55:25 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '0c5173ae57bd4e87b1bb6c2c2348f095', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/50
140/140 [==============================] - 257s 1s/step - loss: 138.9264 - val_loss: nan
Epoch 2/50
140/140 [==============================] - 132s 945ms/step - loss: 108.4297 - val_loss: nan
Epoch 3/50
140/140 [==============================] - 134s 957ms/step - loss: 102.1505 - val_loss: nan
Epoch 4/50
140/140 [==============================] - 132s 947ms/step - loss: 98.6317 - val_loss: nan
Epoch 5/50
140/140 [==============================] - 132s 944ms/step - loss: 96.4973 - val_loss: nan
Epoch 6/50
140/140 [==============================] - 132s 941ms/step - loss: 94.6732 - val_loss: nan
Epoch 7/50
140/140 [==============================] - 133s 947ms/step - loss: 93.4806 - val_loss: nan
Epoch 8/50
140/140 [==============================] - 132s 946ms/step - loss: 92.2484 - val_loss: nan
Epoch 9/50
140/140 [==============================] - 132s 943ms/step - loss: 91.3171 - val_loss: nan
Epoch 10/50
140/140 [==============================] - 133s 947ms/step - loss: 90.

2021/08/11 05:48:26 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "D:\Users\same\anaconda3\lib\site-packages\mlflow\tensorflow.py:805: UserWarning: Logging to MLflow failed: module 'keras.utils.generic_utils' has no attribute 'populate_dict_with_module_objects'"


In [34]:

MODEL_NAME = "deep_augmented"
EPOCHS=20
MINI_BATCH_SIZE = 200
FEATURES_LIST=total_features

TEXT_LIST=total_text

FEATURES_VALID_LIST=total_valid_features

TEXT_VALID_LIST=total_valid_text

audio_gen = Batch(FEATURES_LIST,FEATURES_VALID_LIST,TEXT_LIST,TEXT_VALID_LIST,MINI_BATCH_SIZE)

In [35]:
deep_speech = model_2(input_dim=26,
                filters=100,
                kernel_size=4, 
                conv_stride=1,
                conv_border_mode='valid',
                units=250,
                activation='tanh',
                dropout_rate=0.6,
                number_of_layers=2,
                output_dim=len(char_map)+1)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 26)]        0         
_________________________________________________________________
layer_1_conv (Conv1D)        (None, None, 100)         10500     
_________________________________________________________________
conv_batch_norm (BatchNormal (None, None, 100)         400       
_________________________________________________________________
rnn_1 (GRU)                  (None, None, 250)         264000    
_________________________________________________________________
bt_rnn_1 (BatchNormalization (None, None, 250)         1000      
_________________________________________________________________
final_layer_of_rnn (GRU)     (None, None, 250)         376500    
_________________________________________________________________
bt_rnn_final (BatchNormaliza (None, None, 250)         1000

In [36]:
mlflow.tensorflow.autolog()
mlflow.set_experiment("Deep Speech Augmented")
train(audio_gen, input_to_softmax=deep_speech, model_name=MODEL_NAME, epochs=EPOCHS, minibatch_size=MINI_BATCH_SIZE)

mlflow.end_run()

2021/08/11 05:48:32 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '8a646302c2774c5fb308a379f3ae2889', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/20
140/140 [==============================] - 118s 799ms/step - loss: 145.2840 - val_loss: 131.5580
Epoch 2/20
140/140 [==============================] - 110s 783ms/step - loss: 107.1035 - val_loss: 110.4479
Epoch 3/20
140/140 [==============================] - 110s 785ms/step - loss: 97.6631 - val_loss: 104.8024
Epoch 4/20
140/140 [==============================] - 111s 790ms/step - loss: 91.6348 - val_loss: 100.0569
Epoch 5/20
140/140 [==============================] - 110s 785ms/step - loss: 87.0408 - val_loss: 99.0599
Epoch 6/20
140/140 [==============================] - 114s 817ms/step - loss: 83.5312 - val_loss: 96.8731
Epoch 7/20
140/140 [==============================] - 123s 878ms/step - loss: 80.5837 - val_loss: 96.6903
Epoch 8/20
140/140 [==============================] - 122s 872ms/step - loss: 77.9169 - val_loss: 98.4714
Epoch 9/20
140/140 [==============================] - 122s 869ms/step - loss: 75.7535 - val_loss: 92.8637
Epoch 10/20
140/140 [===================